In [1]:
import sys
sys.path.append('..')

from utils.DatabaseUtils import SQLLiteUtils
from utils.FileSystemUtils import FileSystemUtils
from consts.const_variables import *
from utils.common_utils import *
from utils.setup import set_me_up
from time import time
from algorithms.rock import Rock
from algorithms.clope import Clope
from algorithms.outlier_detection_algorithms import *
from algorithms.edm_cif import EDM_CIF
from utils.data_objects import *
from collections import Counter
import os
import warnings
warnings.filterwarnings("ignore")
os.chdir("..")
set_me_up()

db_session = SQLLiteUtils(database)
fs_utils = FileSystemUtils('data')
datasets, datasets_class, ohe_datasets = read_datasets_to_dict(db_session)

Working Directory Set To: C:\Users\imagi\PycharmProjects\edm-cif


In [2]:
datasets['real'].keys()

dict_keys(['agaricus-lepiota', 'breast-cancer', 'balance-scale', 'car', 'house-votes-84', 'nursery', 'soybean-large', 'NPHA-doctor-visits', 'flare1', 'flare2', 'primary-tumor', 'SPECT', 'promoters', 'monks', 'lymphoraphy', 'splice', 'hiv_protease_cleavage', 'Interests_group', 'diamonds', 'bike_buyers'])

In [3]:
################### TEST ROCK ############################

name = 'agaricus-lepiota'
dataset_type = 'real'
df = datasets[dataset_type][name]
df_ohe = ohe_datasets[dataset_type][name]
class_ = datasets_class[dataset_type][name]
print('Real classes:', len(set(class_))-1, " Real outliers:", len(class_.loc[class_==-1]))

obj = Rock(eps=0.5, k=20)
t = time()
obj.fit_predict(df.to_numpy())
t = time()-t
labels = obj.labels
print(Counter(int(x) for x in labels))

Real classes: 1  Real outliers: 0
Counter({4: 1728, 9: 1728, 5: 1296, 0: 960, 1: 768, 7: 288, 16: 288, 6: 192, 8: 192, 15: 192, 11: 104, 2: 96, 3: 96, 10: 48, 12: 48, 17: 36, 13: 32, 19: 16, 14: 8, 18: 8})


In [4]:
# wyszukać 0.5%, 2%, 10% w rzeczywistych zbiorach
threshold = 8
result = obj.select_outliers(threshold)
print("All finded:", len(result[result==-1]), " Matches:", np.sum((result == -1) & (class_.to_numpy() == -1)))

All finded: 16  Matches: 0


In [5]:
############### REAL #####################
out_process = OutlierDetectionProcess(algorithm='rock', split_mode=None, split_method=None, dataset=name, dataset_type=dataset_type,
                                      n_estimators=None, max_samples=None, min_cluster_size=None, contamination=None, max_features=None,
                                      alpha=None, beta=None, theta=None, epsilon=obj.eps, k=obj.k, ground_truth=0, clustering=1, time_sec=t,
                                      metric=None, outliers_threshold=threshold, n_neighbors=None, task=None, m=None)


out_process.count_metrics(df, df_ohe, class_.to_numpy(), result)
contrastive_outlier = ContrastiveOutlier(out_process.process_id)
contrastive_outlier.calculate_configurations(df_ohe, result)

In [6]:
out_process

OutlierDetectionProcess(algorithm='rock', dataset='agaricus-lepiota', dataset_type='real', split_mode=None, split_method=None, n_estimators=None, max_samples=None, min_cluster_size=None, contamination=None, n_neighbors=None, max_features=None, alpha=None, beta=None, epsilon=0.5, k=20, m=None, theta=None, metric=None, ground_truth=0, clustering=1, outliers_threshold=8, time_sec=28.321336269378662, task='outliers_detection', detected_outliers_num=16, detected_outliers_perc=0.002, real_outliers_num=None, real_outliers_perc=None, precision=None, recall=None, accuracy=None, tp=None, tn=None, fp=None, fn=None, tpr=None, tnr=None, fpr=None, fnr=None, clusters=18, silhouette_score=0.4712, calinski_harabasz_score=735.1825, davies_bouldin_score=0.3919, dunn_index=0.2727, cluster_entropy=0.4759, fmi=None, ari=None, vi=None, nmi=None, shannon=None, process_id='b3789783e30a4c2fa76a13e25c6cbec8', distribution='{0: 960, 1: 768, 2: 96, 3: 96, 4: 1728, 5: 1296, 6: 192, 7: 288, 8: 192, 9: 1728, 10: 48, 

In [7]:
#db_session.insert_into_table_from_dictionary(outliers_table_name, asdict(out_process))
#contrastive_outlier.insert_to_db(db_session, contrastive_outlier_score_table_name)

In [8]:
################### TEST CLOPE ############################
name = 'bike_buyers'
dataset_type = 'real'
df = datasets[dataset_type][name]
df_ohe = ohe_datasets[dataset_type][name]
class_ = datasets_class[dataset_type][name]
print('Real classes:', len(set(class_))-1, " Real outliers:", len(class_.loc[class_==-1]))

obj = Clope(r=1.2)
t = time()
obj.fit_predict(df.to_numpy())
t = time()-t
labels = obj.labels
print(Counter(int(x) for x in labels))

Real classes: 1  Real outliers: 0
Counter({5: 375, 1: 236, 6: 132, 2: 98, 3: 67, 4: 58, 0: 28, 7: 6})


In [9]:
threshold = 6
result = obj.select_outliers(threshold)
print("All finded:", len(result[result==-1])/len(df), " Matches:", np.sum((result == -1) & (class_.to_numpy() == -1)))

All finded: 0.006  Matches: 0


In [10]:
out_process = OutlierDetectionProcess(algorithm='clope', split_mode=None, split_method=None, dataset=name, dataset_type=dataset_type,
                                      n_estimators=None, max_samples=None, min_cluster_size=None, contamination=None, max_features=None,
                                      alpha=None, beta=None, theta=obj.r, epsilon=None, k=None, ground_truth=0, clustering=1, time_sec=t,
                                      metric=None, outliers_threshold=threshold, n_neighbors=None, m=None, task=None)
out_process.count_metrics(df, df_ohe, class_.to_numpy(), result)
contrastive_outlier = ContrastiveOutlier(out_process.process_id)
contrastive_outlier.calculate_configurations(df_ohe, result)

In [11]:
#db_session.insert_into_table_from_dictionary(outliers_table_name, asdict(out_process))
#contrastive_outlier.insert_to_db(db_session, contrastive_outlier_score_table_name)

In [15]:
################### TEST ISOLATION FOREST, LOF, CBLOF, DBSCAN ############################

name = 'dataset_1'
dataset_type = 'synthetic'
df = datasets[dataset_type][name]
df_ohe = ohe_datasets[dataset_type][name]
class_ = datasets_class[dataset_type][name].astype(int)
print('Real classes:', len(set(class_))-1, " Real outliers:", len(class_.loc[class_==-1]))

# Isolation Forest algorithm
iso, iso_params, iso_labels = HyperIF(df_ohe, 0.02)

# LOF algorithm
lof, lof_params, lof_labels = HyperLOF(df_ohe, 0.02)

# CBLOF algorithm
cblof, cblof_params, cblof_labels = HyperCBLOF(df_ohe, 0.02)

# HDBSCAN algorithm
hdbscan, hdbscan_params, hdbscan_labels = HyperHDBSCAN(df_ohe)

# EDM-CIF algorithm
edm_cif = EDM_CIF(n_estimators=300, max_features=5, max_samples=500, m=2, contamination=0.04)
edm_cif_labels = edm_cif.fit_predict(df)

print("Isolation Forest results")
print(Counter(int(x) for x in iso_labels))
print("All finded:", len(iso_labels[iso_labels==-1])/len(df), " Matches:", np.sum((iso_labels == -1) & (class_.to_numpy() == -1)))

print("LOF results")
print(Counter(int(x) for x in lof_labels))
print("All finded:", len(lof_labels[lof_labels==-1])/len(df), " Matches:", np.sum((lof_labels == -1) & (class_.to_numpy() == -1)))

print("CBLOF results")
print(Counter(int(x) for x in cblof_labels))
print("All finded:", len(cblof_labels[cblof_labels==-1])/len(df), " Matches:", np.sum((cblof_labels == -1) & (class_.to_numpy() == -1)))

print("HDBSCAN results")
print(Counter(int(x) for x in hdbscan_labels))
print("All finded:", len(hdbscan_labels[hdbscan_labels==-1])/len(df), " Matches:", np.sum((hdbscan_labels == -1) & (class_.to_numpy() == -1)))

print("EDM-CIF results")
print(Counter(int(x) for x in edm_cif_labels))
print("All finded:", len(edm_cif_labels[edm_cif_labels==-1])/len(df), " Matches:", np.sum((edm_cif_labels == -1) & (class_.to_numpy() == -1)))

Real classes: 2  Real outliers: 60
Isolation Forest results
Counter({1: 2941, -1: 59})
All finded: 0.019666666666666666  Matches: 38
LOF results
Counter({1: 2940, -1: 60})
All finded: 0.02  Matches: 7
CBLOF results
Counter({0: 2960, -1: 40})
All finded: 0.013333333333333334  Matches: 28
HDBSCAN results
Counter({39: 870, 25: 818, -1: 149, 19: 87, 34: 71, 10: 66, 6: 59, 31: 55, 23: 53, 26: 43, 16: 41, 21: 39, 24: 33, 30: 33, 37: 33, 22: 30, 13: 28, 33: 27, 9: 27, 3: 27, 2: 25, 40: 25, 29: 25, 4: 25, 36: 24, 41: 23, 17: 22, 1: 22, 12: 22, 14: 21, 27: 20, 28: 19, 18: 19, 32: 18, 35: 17, 38: 14, 0: 14, 15: 12, 7: 12, 8: 11, 20: 7, 5: 7, 11: 7})
All finded: 0.049666666666666665  Matches: 33
EDM-CIF results
Counter({1: 2879, -1: 121})
All finded: 0.04033333333333333  Matches: 45


In [ ]:
out_process = OutlierDetectionProcess(algorithm='lof', split_mode=None, split_method=None, dataset=name, dataset_type=dataset_type,
                                      n_estimators=None, max_samples=None, min_cluster_size=None, contamination=iso.contamination, max_features=None,
                                      alpha=None, beta=None, theta=None, epsilon=None, k=None, ground_truth=0, clustering=0, time_sec=t,
                                      metric=iso.metric, outliers_threshold=None, n_neighbors=iso.n_neighbors, m=None, task=None)
out_process.count_metrics(df, df_ohe, class_.to_numpy(), labels)
contrastive_outlier = ContrastiveOutlier(out_process.process_id)
contrastive_outlier.calculate_configurations(df_ohe, labels)
db_session.insert_into_table_from_dictionary(outliers_table_name, asdict(out_process))
contrastive_outlier.insert_to_db(db_session, contrastive_outlier_score_table_name)

In [ ]:
out_process = OutlierDetectionProcess(algorithm='isolation_forest', split_mode=None, split_method=None, dataset=name, dataset_type=dataset_type,
                                      n_estimators=iso.n_estimators, max_samples=iso.max_samples, min_cluster_size=None, contamination=iso.contamination, max_features=iso.max_features,
                                      alpha=None, beta=None, theta=None, epsilon=None, k=None, ground_truth=1, clustering=0, time_sec=t,
                                      metric=None, outliers_threshold=None, n_neighbors=None, m=None, task=None)
out_process.count_metrics(df, df_ohe, class_.to_numpy(), labels)
contrastive_outlier = ContrastiveOutlier(out_process.process_id)
contrastive_outlier.calculate_configurations(df_ohe, labels)